In [25]:
# Variables
source_project = ''
source_dataset = ''
source_table = ''

# In what percentage of events does an event parameter need to be found to be
# considered a universal event parameter. In other words, what parameters should
# be configured for all events.
threshold = 0.7

exclude_parameters = []
# comment out the below line to not exclude any parameters
exclude_parameters = ['page_title', 'batch_ordering_id', 'engaged_session_event', 'batch_page_id', 'page_location',
                      'ignore_referrer', 'gtm_container_id', 'engagement_time_msec', 'gtm_container_version', 'page_referrer',
                      'session_engaged', 'gtm_debug_mode', 'ga_session_number' , 'ga_session_id', 'medium', 'term', 'campaign',
                      'source', 'gclid', 'gclsrc', 'gtm_environment', 'debug_mode']
exclude_parameters = ', '.join(exclude_parameters)

In [ ]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')
%load_ext google.colab.data_table

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)
try:
  sh = gc.open(source_project +':' + source_dataset + ':' + source_table)
except gspread.exceptions.SpreadsheetNotFound: 
  sh = gc.create(source_project +':' + source_dataset + ':' + source_table)

In [ ]:
from google.cloud import bigquery

client = bigquery.Client(project=source_project)

events = client.query('''
  select distinct
    event_name
  from {sp}.{sd}.{st} '''.format(sp=source_project, sd=source_dataset, st=source_table) ).result()

event_list=[]
for event in events:
  event_list.append(event[0])
event_list

In [ ]:
# Dictionary of dataframes
event_params = {}

for event in event_list:
  exclude_list = [f"'{param}'" for param in exclude_parameters.split(', ')]
  event_params[event] = client.query('''
    select
      ep.key
      , count(ep.value.string_value) as string_values
      , count(ep.value.int_value) as int_values
      , count(ep.value.float_value) as float_values
      , count(ep.value.double_value) as double_values
    from {sp}.{sd}.{st}
    cross join
      unnest(event_params) ep
    where event_name = '{en}'
    and ep.key not in ({excl})
    group by ep.key;
    '''.format(sp=source_project, sd=source_dataset, st=source_table, en=event, excl=', '.join(exclude_list)) ).to_dataframe()
  event_params[event]['value_types'] = ""


user_props = client.query('''
  select
    up.key
    , count(case when up.value.string_value is not null then up.value.string_value end) as string_values
    , count(case when up.value.int_value is not null then up.value.int_value end) as int_values
    , count(case when up.value.float_value is not null then up.value.float_value end) as float_values
    , count(case when up.value.double_value is not null then up.value.double_value end) as double_values
  from {sp}.{sd}.{st}
  cross join
    unnest(user_properties) up
  group by up.key;
  '''.format(sp=source_project, sd=source_dataset, st=source_table) ).to_dataframe()


In [ ]:
# warn_params is a list of event and event parameter names that have more than
# one type of value
warn_params = []
warn_props = []

# List parameters that are found in most or all events
param_set = set()
parameter_occurrences = {}
total_events = 0
universal_params = []

# List user properties
prop_set = set()


for event in event_list:
  total_events += 1
  for param in event_params[event].key:
    for index, row in event_params[event].iterrows():
      matched_params = []
      if row['string_values'] > 0:
        matched_params.append('string_value')
      if row['int_values'] > 0:
        matched_params.append('int_value')
      if row['float_values'] > 0:
        matched_params.append('float_value')
      if row['double_values'] > 0:
        matched_params.append('double_value')
      if len(matched_params) > 1:
        warn_params.append( event + ": " + param )
      event_params[event].loc[index, 'value_types'] = ', '.join(matched_params)

      param_set.add(row.key)
      if row['key'] not in parameter_occurrences:
        parameter_occurrences[row['key']] = 1
      else:
        parameter_occurrences[row['key']] += 1

for param in param_set:
  if parameter_occurrences[param] / total_events >  threshold:
    universal_params.append(param)


for index, row in user_props.iterrows():
  matched_props = []
  if row['string_values'] > 0:
    matched_props.append('string_value')
  if row['int_values'] > 0:
    matched_props.append('int_value')
  if row['float_values'] > 0:
    matched_props.append('float_value')
  if row['double_values'] > 0:
    matched_props.append('double_value')
  if len(matched_props) > 1:
    warn_props.append( event + ": " + prop )
  user_props.loc[index, 'value_types'] = ', '.join(matched_props)

  prop_set.add(row.key)


In [ ]:
if len(universal_params) > 0:
  print( "The following event parameters appear in at least " + format(threshold, ".0%") + " of events. Consider implementing these as default custom parameters. \n" )
  print( "Please note that the parameters that are implemented by default in GA4 on all events, like page_location, are not removed from this list. \n" )
  print( "You do not need to register default parameters. \n" )
  print('\n '.join(universal_params) + '\n\n')
  try:
    worksheet = sh.worksheet("Universal params")
  except gspread.exceptions.WorksheetNotFound:
    worksheet = sh.add_worksheet(title="Universal params", rows=len(universal_params)+2, cols=1)
  worksheet.update_acell('A1', 'The following event parameters appear in at least ' + format(threshold, ".0%") + ' of events. Consider implementing these as default custom parameters.')
  worksheet.append_rows([[param] for param in universal_params])

print( "These are the event parameters by event: \n" )
for event in event_list:
  print (event +'\n')
  print (event_params[event].loc[:, ['key','value_types']])
  print ('\n')
  try:
    worksheet = sh.worksheet(event)
  except gspread.exceptions.WorksheetNotFound:
    worksheet = sh.add_worksheet(title=event, rows=len(event_params[event].index)+2, cols=len(event_params[event].columns))
  worksheet.update([event_params[event].columns.values.tolist()] + event_params[event].values.tolist())



print( "These are the user properties that appear in the data: \n" )

print (user_props.loc[:, ['key','value_types']])
try:
  worksheet = sh.worksheet("user_props")
except gspread.exceptions.WorksheetNotFound:
  worksheet = sh.add_worksheet(title="user_props", rows=len(user_props.index)+2, cols=len(user_props.columns))
worksheet.update([user_props.columns.values.tolist()] + user_props.values.tolist())


# Function to determine the most common value type
def get_most_common_value_type(row):
    value_types = []
    counts = []
    
    if row['string_values'] > 0:
        value_types.append('string_value')
        counts.append(row['string_values'])
    if row['int_values'] > 0:
        value_types.append('int_value')
        counts.append(row['int_values'])
    if row['float_values'] > 0:
        value_types.append('float_value')
        counts.append(row['float_values'])
    if row['double_values'] > 0:
        value_types.append('double_value')
        counts.append(row['double_values'])
    
    if not value_types:
        return ""
    
    # Return the value type with the highest count
    return value_types[counts.index(max(counts))]

# Create a dictionary to store parameters with multiple value types
multiple_value_types = {}

# Process event parameters to identify most common value types
for event in event_list:
    for index, row in event_params[event].iterrows():
        if ',' in row['value_types']:  # Multiple value types detected
            most_common = get_most_common_value_type(row)
            event_params[event].loc[index, 'most_common_type'] = most_common
            
            # Store for warning message
            if event not in multiple_value_types:
                multiple_value_types[event] = []
            multiple_value_types[event].append((row['key'], row['value_types'], most_common))
        else:
            event_params[event].loc[index, 'most_common_type'] = row['value_types']

print("The following event parameters appear in at least " + format(threshold, ".0%") + " of events. Consider implementing these as default custom parameters. \n")
print("Please note that the parameters that are implemented by default in GA4 on all events, like page_location, are not removed from this list. \n")
print("You do not need to register default parameters. \n")
print('\n '.join(universal_params) + '\n\n')

# Print universal parameters in dbt-GA4 format
print("# Universal Parameters (found in at least " + format(threshold, ".0%") + " of events)")
print("vars:")
print("  ga4:")
print("    universal_custom_parameters:")
for param in sorted(universal_params):
    # Find this parameter in any event to get its type
    param_type = ""
    for event in event_list:
        param_row = event_params[event][event_params[event]['key'] == param]
        if not param_row.empty:
            if 'most_common_type' in param_row.columns:
                param_type = param_row['most_common_type'].values[0]
            else:
                param_type = param_row['value_types'].values[0]
            break
    
    print(f'      - name: "{param}"')
    print(f'        value_type: {param_type}')

print("\n# Event-specific Parameters")
print("vars:")
print("  ga4:")

# Sort events alphabetically
for event in sorted(event_list):
    # Skip if no parameters
    if event_params[event].empty:
        continue
        
    print(f"    {event}_custom_parameters:")
    
    # Sort parameters alphabetically by key
    sorted_params = event_params[event].sort_values('key')
    
    for index, row in sorted_params.iterrows():
        param_name = row['key']
        
        # Skip universal parameters as they're already defined
        if param_name in universal_params:
            continue
            
        value_type = row['most_common_type'] if 'most_common_type' in row else row['value_types']
        
        print(f'      - name: "{param_name}"')
        print(f'        value_type: {value_type}')
        
        # Add warning comment if multiple value types detected
        if ',' in row['value_types']:
            print(f'        # WARNING: Multiple value types detected ({row["value_types"]}). Using most common: {value_type}')

print("\n# User Properties")
print("vars:")
print("  ga4:")
print("    user_properties:")

# Collect all unique user properties across events
all_user_props = {}
for event in user_props:
    if user_props[event].empty or user_props[event]['value_types'].isnull().all():
        continue
        
    for index, row in user_props[event].iterrows():
        prop_name = row['key']
        if prop_name not in all_user_props:
            # Determine most common value type
            most_common = get_most_common_value_type(row)
            all_user_props[prop_name] = {
                'value_types': row['value_types'],
                'most_common': most_common
            }

# Print user properties in alphabetical order
for prop_name in sorted(all_user_props.keys()):
    prop_info = all_user_props[prop_name]
    print(f'      - name: "{prop_name}"')
    print(f'        value_type: {prop_info["most_common"]}')
    
    # Add warning comment if multiple value types detected
    if ',' in prop_info['value_types']:
        print(f'        # WARNING: Multiple value types detected ({prop_info["value_types"]}). Using most common: {prop_info["most_common"]}')

# Print summary of parameters with multiple value types
if multiple_value_types:
    print("\n# WARNING: The following parameters have multiple value types and should be fixed at the source:")
    for event in sorted(multiple_value_types.keys()):
        print(f"\n## Event: {event}")
        for param_info in sorted(multiple_value_types[event], key=lambda x: x[0]):
            param_name, value_types, most_common = param_info
            print(f"  - Parameter: {param_name}")
            print(f"    Found types: {value_types}")
            print(f"    Using: {most_common}")


In [ ]:
# Import pandas
import pandas as pd

# Function to determine the most common value type
def get_most_common_value_type(row):
    value_types = []
    counts = []
    
    if row['string_values'] > 0:
        value_types.append('string_value')
        counts.append(row['string_values'])
    if row['int_values'] > 0:
        value_types.append('int_value')
        counts.append(row['int_values'])
    if row['float_values'] > 0:
        value_types.append('float_value')
        counts.append(row['float_values'])
    if row['double_values'] > 0:
        value_types.append('double_value')
        counts.append(row['double_values'])
    
    if not value_types:
        return ""
    
    # Return the value type with the highest count
    return value_types[counts.index(max(counts))]

# Function to normalize event names according to dbt-GA4 conventions
def normalize_event_name(event_name):
    # Convert to lowercase and replace spaces and hyphens with underscores
    return event_name.lower().replace(' ', '_').replace('-', '_')

# Create a dictionary to store parameters with multiple value types
multiple_value_types = {}

# Process event parameters to identify most common value types
for event in event_list:
    for index, row in event_params[event].iterrows():
        if ',' in row['value_types']:  # Multiple value types detected
            most_common = get_most_common_value_type(row)
            event_params[event].loc[index, 'most_common_type'] = most_common
            
            # Store for warning message
            if event not in multiple_value_types:
                multiple_value_types[event] = []
            multiple_value_types[event].append((row['key'], row['value_types'], most_common))
        else:
            event_params[event].loc[index, 'most_common_type'] = row['value_types']

print("The following event parameters appear in at least " + format(threshold, ".0%") + " of events. Consider implementing these as default custom parameters. \n")
print("Please note that the parameters that are implemented by default in GA4 on all events, like page_location, are not removed from this list. \n")
print("You do not need to register default parameters. \n")
print('\n '.join(universal_params) + '\n\n')

# Print universal parameters in dbt-GA4 format
print("# Universal Parameters (found in at least " + format(threshold, ".0%") + " of events)")
print("vars:")
print("  ga4:")
print("    universal_custom_parameters:")
for param in sorted(universal_params):
    # Find this parameter in any event to get its type
    param_type = ""
    for event in event_list:
        param_row = event_params[event][event_params[event]['key'] == param]
        if not param_row.empty:
            if 'most_common_type' in param_row.columns:
                param_type = param_row['most_common_type'].values[0]
            else:
                param_type = param_row['value_types'].values[0]
            break
    
    print(f'      - name: "{param}"')
    print(f'        value_type: {param_type}')

print("\n# Event-specific Parameters")
print("vars:")
print("  ga4:")

# Create a list of normalized event names for sorting
normalized_events = [(event, normalize_event_name(event)) for event in event_list]
# Sort by the normalized (lowercase) event name
normalized_events.sort(key=lambda x: x[1])

# Process events in alphabetical order (using normalized names)
for original_event, normalized_event in normalized_events:
    # Skip if no parameters
    if event_params[original_event].empty:
        continue
        
    print(f"    {normalized_event}_custom_parameters:")
    
    # Sort parameters alphabetically by key
    sorted_params = event_params[original_event].sort_values('key')
    
    for index, row in sorted_params.iterrows():
        param_name = row['key']
        
        # Skip universal parameters as they're already defined
        if param_name in universal_params:
            continue
            
        value_type = row['most_common_type'] if 'most_common_type' in row else row['value_types']
        
        print(f'      - name: "{param_name}"')
        print(f'        value_type: {value_type}')
        
        # Add warning comment if multiple value types detected
        if ',' in row['value_types']:
            print(f'        # WARNING: Multiple value types detected ({row["value_types"]}). Using most common: {value_type}')

# Check if user properties exist and have been processed
has_user_props = False
valid_user_props = {}

# First, safely check which user_props are valid and have data
for event in list(user_props.keys()):
    # Check if the DataFrame exists and is not empty
    if event in user_props and isinstance(user_props[event], pd.DataFrame) and not user_props[event].empty:
        # Check if 'value_types' column exists
        if 'value_types' in user_props[event].columns:
            # Check if there are non-null values
            if not user_props[event]['value_types'].isnull().all():
                has_user_props = True
                valid_user_props[event] = user_props[event]

if has_user_props:
    print("\n# User Properties")
    print("vars:")
    print("  ga4:")
    print("    user_properties:")

    # Collect all unique user properties across events
    all_user_props = {}
    for event in valid_user_props:
        for index, row in valid_user_props[event].iterrows():
            prop_name = row['key']
            if prop_name not in all_user_props:
                # Determine most common value type
                most_common = get_most_common_value_type(row)
                all_user_props[prop_name] = {
                    'value_types': row['value_types'],
                    'most_common': most_common
                }

    # Print user properties in alphabetical order
    for prop_name in sorted(all_user_props.keys()):
        prop_info = all_user_props[prop_name]
        print(f'      - name: "{prop_name}"')
        print(f'        value_type: {prop_info["most_common"]}')
        
        # Add warning comment if multiple value types detected
        if ',' in prop_info['value_types']:
            print(f'        # WARNING: Multiple value types detected ({prop_info["value_types"]}). Using most common: {prop_info["most_common"]}')
else:
    print("\n# No User Properties Found")

# Print summary of parameters with multiple value types
if multiple_value_types:
    print("\n# WARNING: The following parameters have multiple value types and should be fixed at the source:")
    
    # Sort events by normalized name for the warning section too
    normalized_warning_events = [(event, normalize_event_name(event)) for event in multiple_value_types.keys()]
    normalized_warning_events.sort(key=lambda x: x[1])
    
    for original_event, normalized_event in normalized_warning_events:
        print(f"\n## Event: {normalized_event} (original: {original_event})")
        for param_info in sorted(multiple_value_types[original_event], key=lambda x: x[0]):
            param_name, value_types, most_common = param_info
            print(f"  - Parameter: {param_name}")
            print(f"    Found types: {value_types}")
            print(f"    Using: {most_common}")
